### 1) IMPORTING PACKAGES AND DATA

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

### 2) CLEANING DATA

In [ ]:

train.isnull().sum()


#combin sipsp and parch to family then drop
train['Combine'] = train['SibSp'] +train['Parch']
test['Combine'] = test['SibSp'] +test['Parch']


#too many cabin null values so I'll just drop
#ticket is also irelevant 

train = train.drop(['Cabin','Ticket','Name','SibSp','Parch'], axis = 1)
test = test.drop(['Cabin','Ticket','Name','SibSp','Parch'], axis = 1)

In [ ]:
train['Age'] = train['Age'].fillna(train['Age'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())
train['Embarked'] = train['Embarked'].fillna('S')

test['Fare'] = test['Fare'].fillna(test['Fare'].median())

In [ ]:
test.isnull().sum()

### 3) DATA VISUALIZATION

In [ ]:
train.head()

In [ ]:
sns.pairplot(train, hue ='Survived')

From the pairplot, it seems like multiclass classification is not efficient in determining the survival rate

In [ ]:
sns.histplot(data = train, x = 'Age', hue = 'Survived', bins = 5,
            kde = True, stat = 'percent')

In [ ]:
#set female to 1 and male to 0 for train data
train['Sex'] = train['Sex'].map({'male':0, 'female':1}).astype(int)

#set female to 1 and male to 0 for test data
test['Sex'] = test['Sex'].map({'male':0, 'female':1}).astype(int)

In [ ]:
#set embarked train and test to 1,2,3

train['Embarked'] = train['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)
test['Embarked'] = test['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)

In [ ]:
x = train.drop('Survived',axis =1)
y = train['Survived']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(x,y)

#create logistic regression object

log_model = LogisticRegression(max_iter = 1000)
log_model.fit(x_train,y_train)


prediction = log_model.predict(x_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,prediction)

### Using KNN Algorithm

In [ ]:
X_train = train.drop(['PassengerId','Survived'], axis = 1)
Y_train = train['Survived']

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


#apply to training dataset

knn_pred = KNeighborsClassifier(n_neighbors = 3)

knn_pred.fit(X_train,Y_train)

y_pred_knn = knn_pred.predict(x_test.drop('PassengerId',axis=1))
print(metrics.accuracy_score(y_test,y_pred_knn))


In [ ]:
#cycle to different k values

k_range = [1,100]

accuracy = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train,Y_train)
    
    y_pred_knn = knn.predict(x_test.drop('PassengerId',axis = 1))
    accuracy.append(metrics.accuracy_score(y_test, y_pred_knn))

plt.plot(k_range,accuracy)

In [ ]:
submission_pred = knn_pred.predict(test.drop('PassengerId',axis = 1))

df = pd.DataFrame({'PassengerId':test.PassengerId,
                 'Survived':submission_pred})

In [ ]:
df.to_csv('Submission.csv', index = False)